## i) Load Libraries
### The first step in the data processing is to simply load in all of the libraries required.

In [1]:
#load all required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt 
from pandas.io.json import json_normalize

### ii) Load Data
The next step is to load the data into the notebook and briefly inspect it. This includes both the CSV files and the JSON files. 

In [8]:
#%cd path

In [9]:
youtubeUS = pd.read_csv("USvideos.csv") #load US data using raw string

In [10]:
youtubeUS.shape #check dataframe dimensions

(40949, 16)

In [11]:
youtubeGB = pd.read_csv("GBvideos.csv") #load Great Britain youtube data 

In [12]:
youtubeGB.shape #check dataframe dimensions

(38916, 16)

In [13]:
youtubeIN = pd.read_csv("INvideos.csv") #load India youtube data 

In [14]:
youtubeIN.shape #check dataframe dimensions

(37352, 16)

In [15]:
youtubeUS.head(3) #check dataframe column values to get a feel of data

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...


In [16]:
us_json = pd.read_json("US_category_id.json")

In [17]:
in_json = pd.read_json("IN_category_id.json")

In [18]:
gb_json = pd.read_json("GB_category_id.json")

In [19]:
us_json.head(3) #loading json files

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [20]:
us_json.dtypes

kind     object
etag     object
items    object
dtype: object

### iii) Merging the Datasets
The next step is to merge the datasets for the three countries. To do this, we will first create a tag for each country and then merge the dataframes so each entry can still be identified by country.

In [21]:
youtubeUS.insert(0,"Country","US", True)
youtubeGB.insert(0,"Country","GB", True) #insert country column with country code
youtubeIN.insert(0,"Country","IN", True)

In [22]:
youtubeUSGBIN = pd.concat([youtubeUS, youtubeGB, youtubeIN]) #concat all three dataframes

In [23]:
youtubeUSGBIN.shape #shape of master dataframe

(117217, 17)

### iv) Verify Data Integrity
To verify the integrity of our data, we will check if there are any null entries in our data and if the datatypes match what we want. As can be seen below, the only row we have to with nulls is description meaning our data is relatively clean. This makes sense however since some videos simply do not have descriptions. In terms of data types, that also matches what we want with the exception of the date columns which we want to convert to datetime objects. Lastly, we drop any completely duplicated rows.

In [24]:
youtubeUSGBIN.isnull().any() #description col has some nulls

Country                   False
video_id                  False
trending_date             False
title                     False
channel_title             False
category_id               False
publish_time              False
tags                      False
views                     False
likes                     False
dislikes                  False
comment_count             False
thumbnail_link            False
comments_disabled         False
ratings_disabled          False
video_error_or_removed    False
description                True
dtype: bool

In [25]:
youtubeUSGBIN.isna().sum() #count of nulls

Country                      0
video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               1743
dtype: int64

In [26]:
youtubeUSGBIN.dtypes

Country                   object
video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

In [27]:
origLen = len(youtubeUSGBIN)
df = youtubeUSGBIN.drop_duplicates().copy()
numDropped = origLen - len(df)
print('Dropped ' + str(numDropped) + ' rows.')

Dropped 4482 rows.


### v) Merging json Files With the Dataframe 
The data about categories comes from the json files. In order for the category information to be meaningful for us, we will use the json data to convert the category_id column to be the actual categories themselves.

In [28]:
#getting the ids and categories from India
recs=in_json['items']
in_cat=json_normalize(recs)[['id','snippet.title']]
#getting the ids and categories from USA
recs_us=us_json['items']
us_cat=json_normalize(recs_us)[['id','snippet.title']]
#getting the ids and categories from GB
recs_gb=gb_json['items']
gb_cat=json_normalize(recs_gb)[['id','snippet.title']]
#merging indian and USA categories
categories=pd.merge(in_cat,us_cat,on='id',how='outer')
#merging categories with GB categories
categories=pd.merge(categories,gb_cat,on='id',how='outer')
#dropping unnecessary columns
categories=categories.drop(['snippet.title_x','snippet.title'],axis=1).rename(columns={'snippet.title_y':'category'})

In [29]:
categories.dtypes

id          object
category    object
dtype: object

In [30]:
#converting id column of categories to int to merge with the original dataframe
categories['id']=categories['id'].astype('int')

In [31]:
#adding categories column to the main dataframe "df"
df=pd.merge(left=df,right=categories,left_on='category_id',right_on='id')

In [32]:
#dropping id column from main dataframe
df=df.drop('id',axis=1)

In [33]:
# getting a view of the dataframe to verify it was done correctly
df.head(2)

,Country,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
0,US,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,US,0mlNzVSJrT0,17.14.11,Me-O Cats Commercial,Nobrand,22,2017-04-21T06:47:32.000Z,"cute|""cats""|""thai""|""eggs""",98966,2486,184,532,https://i.ytimg.com/vi/0mlNzVSJrT0/default.jpg,False,False,False,Kittens come out of the eggs in a Thai commerc...,People & Blogs


### vi) Creating a new variable "likeability ratio"
We created a new variable called "likeability ratio" which is the ratio of the number of likes to the number of dislikes. To analyse the hypothesis that does more likes account for more views, in order to achieve that we are taking into account likeability ratio.

In [34]:
df['likeability_ratio']=df['likes']/df['dislikes'].replace(0,1)
#NOTE: for channel_title such '9-1-1 on FOX', '90s Commercials', there are zero dislikes. We need to filter 
#such records and substitute 1 for 0